In [1]:
import multiprocessing
import time
from ray.tune.registry import register_env
from pycigar.utils.registry import make_create_env
from pycigar.utils.input_parser import input_parser
import numpy as np
import tensorflow as tf
import pickle
import matplotlib.pyplot as plt
from pycigar.utils.logging import logger
import os
import pycigar

PATH = os.getcwd()


file_name = 'policy'
policy = os.path.join(PATH, file_name) #'/home/toanngo/Documents/GitHub/ceds-cigar/pycigar/notebooks/policy_eval_old-obs_P30_pset02_35'
start = 100
percentage_hack = 0.45

"""
Load the scenarios configuration file. This file contains the scenario information
for the experiment.
"""
misc_inputs_path = pycigar.DATA_DIR + "/ieee37busdata/misc_inputs.csv"
dss_path = pycigar.DATA_DIR + "/ieee37busdata/ieee37.dss"
load_solar_path = pycigar.DATA_DIR + "/ieee37busdata/load_solar_data.csv"
breakpoints_path = pycigar.DATA_DIR + "/ieee37busdata/breakpoints.csv"

sim_params = input_parser(misc_inputs_path, dss_path, load_solar_path, breakpoints_path, benchmark=True, percentage_hack=percentage_hack)
pycigar_params = {"exp_tag": "cooperative_multiagent_ppo",
                    "env_name": "CentralControlPVInverterEnv",
                    "simulator": "opendss"}

create_env, env_name = make_create_env(pycigar_params, version=0)
register_env(env_name, create_env)
sim_params['scenario_config']['start_end_time'] = [start, start + 750]
del sim_params['attack_randomization']
test_env = create_env(sim_params)
test_env.observation_space  # get the observation space, we need this to construct our agent(s) observation input
test_env.action_space  # get the action space, we need this to construct our agent(s) action output
tf.compat.v1.enable_eager_execution()
policy = tf.saved_model.load(policy)
infer = policy.signatures['serving_default']
done = False
obs = test_env.reset()
obs = obs.tolist()
while not done:
    act_logits = infer(
        prev_reward=tf.constant([0.], tf.float32),
        observations=tf.constant([obs], tf.float32),
        is_training=tf.constant(False),
        seq_lens=tf.constant([0], tf.int32),
        prev_action=tf.constant([0], tf.int64)
    )['behaviour_logits'].numpy()
    act = np.argmax(act_logits)
    obs, r, done, _ = test_env.step(act)
    obs = obs.tolist()
log_dict = logger().log_dict
custom_metrics = logger().custom_metrics

In [4]:
print(log_dict['s701a'].keys())
print(log_dict['inverter_s701a'].keys())
print(log_dict['adversary_inverter_s701a'].keys())

dict_keys(['p', 'q', 'kw', 'kvar', 'voltage'])
dict_keys(['y', 'u', 'p_set', 'q_set', 'p_out', 'q_out', 'control_setting', 'sbar_solarirr', 'sbar_pset', 'solar_irr', 'node', 'reward'])
dict_keys(['y', 'u', 'p_set', 'q_set', 'p_out', 'q_out', 'control_setting', 'sbar_solarirr', 'sbar_pset', 'solar_irr', 'node'])


In [6]:
result = {}
result['v'] = []
result['y'] = []
result['q_set'] = []
result['q_val'] = []
result['a1'] = []
result['a2'] = []
result['a3'] = []
result['a4'] = []
result['a5'] = []
result['a1_adv'] = []
result['a2_adv'] = []
result['a3_adv'] = []
result['a4_adv'] = []
result['a5_adv'] = []

node = 's701a'
for i in range(len(log_dict[node]['voltage'])):
    result['v'].append(log_dict[node]['voltage'][i])
    result['y'].append(log_dict['inverter_' + node]['y'][i])
    result['q_set'].append(log_dict['inverter_' + node]['q_set'][i])
    result['q_val'].append(log_dict['inverter_' + node]['q_out'][i])
    result['a1'].append(log_dict['inverter_' + node]['control_setting'][i][0])
    result['a2'].append(log_dict['inverter_' + node]['control_setting'][i][1])
    result['a3'].append(log_dict['inverter_' + node]['control_setting'][i][2])
    result['a4'].append(log_dict['inverter_' + node]['control_setting'][i][3])
    result['a5'].append(log_dict['inverter_' + node]['control_setting'][i][4])
    result['a1_adv'].append(log_dict['adversary_inverter_' + node]['control_setting'][i][0])
    result['a2_adv'].append(log_dict['adversary_inverter_' + node]['control_setting'][i][1])
    result['a3_adv'].append(log_dict['adversary_inverter_' + node]['control_setting'][i][2])
    result['a4_adv'].append(log_dict['adversary_inverter_' + node]['control_setting'][i][3])
    result['a5_adv'].append(log_dict['adversary_inverter_' + node]['control_setting'][i][4])

In [ ]:
import pandas as pd

df = pd.DataFrame.from_dict(result)
df.to_csv("eval_150_450.csv")